# Logistic Regression

**PyTorch Model Steps**
1. Design model (input, output, forward pass)
2. Construct loss and optimizer
3. Training loop
    * Forward pass: compute prediction and loss
    * Backward pass: gradients
    * Update weights

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Prepare Data

### Load Data

In [2]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_sample, n_feature = X.shape
print(f'n_sample: {n_sample}, n_feature: {n_feature}')

n_sample: 569, n_feature: 30


### Train Test Split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

### Data Scaling

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Convert to Tensor

In [5]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

### Reshape y

In [6]:
y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)

## Model

In [7]:
class LogisticRegression(nn.Module):
    def __init__(self, n_feature):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [9]:
model = LogisticRegression(n_feature)

## Loss & Optimizer

In [21]:
learning_rate = 0.004
criterion = nn.BCELoss()
# optimiizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimiizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training Loop

In [22]:
epochs = 100
for epoch in range(epochs):
    # Forward Pass & Loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    
    # Backward Pass
    loss.backward()
    
    # Update Parameters
    optimiizer.step()
    optimiizer.zero_grad()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch+1}, Loss: {loss.item():.4f}')

Epoch: 10, Loss: 0.0242
Epoch: 20, Loss: 0.0238
Epoch: 30, Loss: 0.0234
Epoch: 40, Loss: 0.0231
Epoch: 50, Loss: 0.0227
Epoch: 60, Loss: 0.0224
Epoch: 70, Loss: 0.0221
Epoch: 80, Loss: 0.0218
Epoch: 90, Loss: 0.0215
Epoch: 100, Loss: 0.0212


## Evaluation

In [20]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum().float() / y_test.shape[0]
    print(f'Accuracy: {acc.item():.4f}')

Accuracy: 0.9649
